In [1]:
import sys
sys.path.append("/home/sergio/TinyYOLOv3-Pedestrian-Detection")

from YOLOblocks import TinyYOLOv3,BasicBlock,PredictionLayer#,YOLOLossBasicBlock
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
#from tensorflow.python.tools import freeze_graph
#from skimage.io import imread,imshow
#from skimage.transform import resize 
import time
#from tensorflow.compat.v1.image import decode_image
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import os

## Dataset preparation

In [2]:
image_feature_description = {
    'bboxes': tf.io.FixedLenFeature([], tf.string),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'num_real_boxes':tf.io.FixedLenFeature([], tf.int64),
}

In [3]:
def get_iou_matrix_tf(box_arr1, box_arr2):
    
    box_arr1 = box_arr1 -tf.tile(box_arr1[:,:2],[1,2])
    #print(box_arr1)
    x11, y11, x12, y12 = tf.split(box_arr1, 4, axis=1)
    x21, y21, x22, y22 = tf.split(box_arr2, 4, axis=1)
    xA = tf.maximum(x11, tf.transpose(x21))
    yA = tf.maximum(y11, tf.transpose(y21))
    xB = tf.minimum(x12, tf.transpose(x22))
    yB = tf.minimum(y12, tf.transpose(y22))
    interArea = tf.maximum((xB - xA + 1e-9), 0) * tf.maximum((yB - yA + 1e-9), 0)
    boxAArea = (x12 - x11 + 1e-9) * (y12 - y11 + 1e-9)
    boxBArea = (x22 - x21 + 1e-9) * (y22 - y21 + 1e-9)
    iou = interArea / (boxAArea + tf.transpose(boxBArea) - interArea)
    return iou,tf.argmax(iou,axis=1)#[:,tf.newaxis]


def fill_yolo_output(boxes,grid_size,num_anchors,which_anchor_box,which_anchor_box_index):
    #print(boxes.shape)
    #noobj_mask = tf.ones((1,grid_size*grid_size*num_anchors))
    #print(noobj_mask.shape)
    
    x_min,y_min,x_max,y_max =tf.split(boxes,4,axis=1)

    #Transforma las coordenadas de (xmin,ymin,xmax,ymax) --> (xcenter,ycenter,width,height)
    width = x_max-x_min
    height = y_max-y_min
    x_global =x_min + tf.math.divide(x_max - x_min,2)
    y_global =y_min + tf.math.divide(y_max - y_min,2)
    
    
    x_min_anchor,y_min_anchor,x_max_anchor,y_max_anchor =tf.split(which_anchor_box,4,axis=1)
    
    width_anchor = x_max_anchor-x_min_anchor
    height_anchor = y_max_anchor-y_min_anchor
    x_global_anchor =x_min_anchor + tf.math.divide(x_max_anchor - x_min_anchor,2)
    y_global_anchor =y_min_anchor + tf.math.divide(y_max_anchor - y_min_anchor,2)   

    
    #print("el x original",x_global)
    #print("el y original",y_global)
    #print("el w original",width)
    #print("el h original",height)
    
    #porción de la imagen que hay en cada celda
    pixel_per_grid = tf.math.divide(1.,grid_size)
    #print(pixel_per_grid)
    
    #Obtenemos la coordenada de la celda donde están los boundingboxes
    offset_grid_x = x_global//pixel_per_grid
    offset_grid_y = y_global//pixel_per_grid
    
    #Obtenemos el el centro locacon referencia  al celda encontrada previamente
    x_local =tf.math.floormod(x_global,pixel_per_grid)
    y_local =tf.math.floormod(y_global,pixel_per_grid)
    #print(x_local,y_local)
    
    #Valores tx e ty del groudtruth
    tx = tf.math.log(x_local + 1e-07/(1-x_local))
    ty = tf.math.log(y_local+1e-07/(1-y_local))
    tw = tf.math.log(tf.math.divide(width+1e-07,width_anchor))
    th = tf.math.log(tf.math.divide(height+1e-07,height_anchor))
    tobj_mask = tf.ones_like(tx)
    tobj = tf.concat([tobj_mask,tobj_mask],axis=0)
    
    #tnoobj = tf.zeros_like(tx)    
    #tobj = tf.ones((grid_size*grid_size*num_anchors,1))
    #tnoobj = tf.zeros((grid_size*grid_size*num_anchors,1))
    #print("Lo que la red debe predecir",tx.numpy(),ty.numpy(),tw.numpy(),th.numpy())
    #x_global = (offset_grid_x * pixel_per_grid) + tf.math.sigmoid(tx)
    #y_global = (offset_grid_y * pixel_per_grid) + tf.math.sigmoid(ty)
    #w = width_anchor*tf.math.exp(tw)
    #h = height_anchor*tf.math.exp(th)
    #print("obtnemos el x_real",x_global)
    #print("obtenemos el y_real",y_global)
    #print("obtenemos el w real",w)
    #print("obtenemos el h real",h)
    
    
    #print(which_anchor_box_index)
    residuo = tf.math.floormod(which_anchor_box_index,2)[:,tf.newaxis]
    coord = tf.cast(2*(offset_grid_y*grid_size + offset_grid_x),dtype=tf.int64)
    
    coord_objectness = tf.cast(2*(offset_grid_y*grid_size + offset_grid_x),dtype=tf.int64)
    coord_objectness2 = coord_objectness+1
    coord_objectess_global = tf.concat([coord_objectness,coord_objectness2],axis=0)
    
    output_position = residuo+coord
    print("tipo de aoutput_positivon",output_position)
    
    print(output_position)
    
    dense_shape = grid_size*grid_size*num_anchors
    tx_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=tx[:,0], dense_shape=[dense_shape]))
    ty_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=ty[:,0], dense_shape=[dense_shape]))
    tw_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=tw[:,0], dense_shape=[dense_shape]))
    th_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=th[:,0], dense_shape=[dense_shape]))
    obj_mask = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=tobj_mask[:,0], dense_shape=[dense_shape]))
    objectness_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=coord_objectess_global, values=tobj[:,0], dense_shape=[dense_shape]))
    #noobj_mask = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=tnoobj[:,0], dense_shape=[dense_shape]))
    #obj_mask =tx_vector=ty_vector=tw_vector=th_vector = tf.zeros((1,grid_size*grid_size*num_anchors))
    
    tx_vector_dense = tf.sparse.to_dense(tx_vector, default_value=0, validate_indices=False, name="Dense_tx")
    ty_vector_dense = tf.sparse.to_dense(ty_vector, default_value=0, validate_indices=False, name="Dense_ty")
    tw_vector_dense = tf.sparse.to_dense(tw_vector, default_value=0, validate_indices=False, name="Dense_tw")
    th_vector_dense = tf.sparse.to_dense(th_vector, default_value=0, validate_indices=False, name="Dense_th")
    obj_mask_dense =  tf.sparse.to_dense(obj_mask, default_value=0, validate_indices=False, name="Dense_obj")
    #noobj_mask_dense = 1-obj_mask_dense
    objectness_vector_dense =  tf.sparse.to_dense(objectness_vector, default_value=0, validate_indices=False)
    
    #noobj_mask_dense= tf.sparse.to_dense(noobj_mask, default_value=1, validate_indices=False, name="Dense_noobj")
    ##print(tx_vector.to_dense)
    #print(tf.sparse.to_dense(tx_vector, default_value=0, validate_indices=True, name=None)
    #tx_vector=tx_vector[[3,2],]
    #tx_vector[output_position[:,0]] = tx
    #print("coordenada de la salida:",output_position)
    
    #return ((tx_vector_dense,ty_vector_dense,obj_mask_dense),(tw_vector_dense,th_vector_dense,obj_mask_dense),(objectness),(objectness))
    
    return tx_vector_dense,ty_vector_dense,tw_vector_dense,th_vector_dense,obj_mask_dense,objectness_vector_dense

def build_targets(image,image_bboxes,num_real_boxes,anchor_boxes):
    
    images_bboxes_original = image_bboxes
    #Obtenemos los boduing boxes que son reales
    image_bboxes = image_bboxes[:num_real_boxes,:]
    #print("Bouding boxes de la imagen",image_bboxes)
    #Obteneos  la matriz de IoU , y el índice del anchor box que dió mejor resultado
    
    image_bboxes = tf.math.divide(image_bboxes,416)
    iou_matrix,which_anchor_box_index = get_iou_matrix_tf(image_bboxes,anchor_boxes)
    
    print(which_anchor_box_index)

    #Indices de los bouding boxes que irian en cada salida, index_best_ yolo nos dice que bouding boxes de la imagen van a la salida YOLO1,
    #porque su mejor IoU fue con los dos anchor boxes mas grandes
    index_best_yolo1 = tf.where(which_anchor_box_index>=2)[:,0]
    index_best_yolo2 = tf.where(which_anchor_box_index<2)[:,0]
    index_best_anchor_yolo1 = tf.gather(which_anchor_box_index,index_best_yolo1,axis=0)
    index_best_anchor_yolo2 = tf.gather(which_anchor_box_index,index_best_yolo2,axis=0)
    
    print(index_best_yolo1)
    print(index_best_anchor_yolo1)

    print(index_best_yolo2)
    print(index_best_anchor_yolo2)

    
    best_bboxes_yolo1 = tf.gather(image_bboxes,index_best_yolo1,axis =0)
    best_anchors_yolo1 = tf.gather(anchor_boxes,index_best_anchor_yolo1, axis =0) #LOs dos anchor boxes grandes corrsponden a YOLO
    best_bboxes_yolo2 = tf.gather(image_bboxes,index_best_yolo2,axis =0)
    best_anchors_yolo2 = tf.gather(anchor_boxes,index_best_anchor_yolo2, axis =0) #Los dos anchor boxes pequeños corresponden a YOLO2
    
    
    if best_anchors_yolo1.shape[0] !=0:
        tx_vector_yolo1,ty_vector_yolo1,tw_vector_yolo1,th_vector_yolo1,obj_mask_yolo1,obj_vector_yolo1= fill_yolo_output(best_bboxes_yolo1,13,2,best_anchors_yolo1,index_best_anchor_yolo1)
    else:
        tx_vector_yolo1=ty_vector_yolo1=tw_vector_yolo1=th_vector_yolo1=obj_mask_yolo1= obj_vector_yolo1=tf.zeros((1,grid_size*grid_size*num_anchors))
        #noobj_mask_yolo1 = tf.ones((1,13*13*num_anchors))
    
    if best_anchors_yolo2.shape[0] != 0:
        tx_vector_yolo2,ty_vector_yolo2,tw_vector_yolo2,th_vector_yolo2,obj_mask_yolo2,obj_vector_yolo2 = fill_yolo_output(best_bboxes_yolo2,26,2,best_anchors_yolo2,index_best_anchor_yolo2)
    else:
        tx_vector_yolo2=ty_vector_yolo2=tw_vector_yolo2=th_vector_yolo2=obj_mask_yolo2 = obj_vector_yolo2=tf.zeros((1,grid_size*grid_size*num_anchors))
        #noobj_mask_yolo2 = tf.ones((1,26*26*num_anchors))
        
    tx_vector = tf.concat([tx_vector_yolo1,tx_vector_yolo2],axis=0)[:,tf.newaxis]
    ty_vector = tf.concat([ty_vector_yolo1,ty_vector_yolo2],axis=0)[:,tf.newaxis]
    tw_vector = tf.concat([tw_vector_yolo1,tw_vector_yolo2],axis=0)[:,tf.newaxis]
    th_vector = tf.concat([th_vector_yolo1,th_vector_yolo2],axis=0)[:,tf.newaxis]
    obj_mask = tf.concat([obj_mask_yolo1,obj_mask_yolo2],axis=0)[:,tf.newaxis]
    #noobj_mask = tf.concat([noobj_mask_yolo1,noobj_mask_yolo2],axis=0)[:,tf.newaxis]
    obj_vector = tf.concat([obj_vector_yolo1,obj_vector_yolo2],axis=0)[:,tf.newaxis]
    
    #output = tf.concat([tx_vector,ty_vector,tw_vector,th_vector,obj_mask,noobj_mask,obj_vector],axis=1)
    #images_bboxes_original
    #return image,output
    #Vamos a regresar obj mask que es 1 cuando hay objeto en grid y el anchor box especifico
    return tf.cast(image,tf.float32)/255,(tf.concat([tx_vector,ty_vector,obj_mask],axis=1),tf.concat([tw_vector,th_vector,obj_mask],axis=1),(obj_mask),(obj_mask))

def imgaug_data_augmentation(image,bboxes,num_real_boxes):
    im_shape = image.shape
    bbs = BoundingBoxesOnImage.from_xyxy_array(bboxes*416, shape=(416,416))
    
    policy = np.random.randint(5)
    
    #policy = 2
    if policy == 0:
        
        p = np.random.random()
        if p<=0.6:
            aug = iaa.TranslateX(px=(-60, 60),cval=128)
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
    
        p = np.random.random()
        if p<=0.8:
            aug = iaa.HistogramEqualization()
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
    
    elif policy==1:
        
        p=np.random.random()
        if p<=0.2:
            aug = iaa.TranslateY(px=(int(-0.18*416), int(0.18*416)),cval=128)
            for i in bbs.to_xyxy_array(np.int32)[:num_real_boxes,:]:
                bbox = image[i[1]:i[3],i[0]:i[2]]
                bbox_augmented = aug(image=bbox)
                image[i[1]:i[3],i[0]:i[2]] = bbox_augmented
        
        p=np.random.random()
        if p<=0.8:
            square_size = np.random.randint(48)
            aug = iaa.Cutout(nb_iterations=1, size=square_size/416, squared=True)
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
            
    elif policy==2:
        p=np.random.random()
        if p<=1:
            aug = iaa.ShearY(shear=(int(-0.06*416), int(0.06*416)), order=1, cval=128)
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
            
        p=np.random.random()
        if p<=0.6:
            aug = iaa.TranslateY(px=(int(-0.18*416), int(0.18*416)),cval=128)
            for i in bbs.to_xyxy_array(np.int32)[:num_real_boxes,:]:
                bbox = image[i[1]:i[3],i[0]:i[2]]
                bbox_augmented = aug(image=bbox)
                image[i[1]:i[3],i[0]:i[2]] = bbox_augmented
            
    elif policy==3:
        p=np.random.random()
        if p<=0.6:    
            aug = iaa.Rotate(rotate=(-30, 30), order=1, cval=128)
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs_aug.remove_out_of_image().clip_out_of_image()
        
        p=np.random.random()
        if p<=1:
            aug = iaa.MultiplySaturation((0.54, 1.54))
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
            
    bbs.remove_out_of_image()
    
    return image,np.clip(bbs.to_xyxy_array(np.float32),1,415),num_real_boxes
    
    
def preprocessing(example_proto):
    image_features = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.image.decode_jpeg(image_features['image_raw'],channels = 3)
    image = tf.cast(tf.image.resize(image,size=(416,416)), tf.uint8)
    bboxes =  tf.io.parse_tensor(image_features['bboxes'], out_type=tf.float32)
    
    num_real_boxes = image_features['num_real_boxes']
    return image,bboxes,num_real_boxes

def preprocessing_validation_set(example_proto):
    image_features = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.image.decode_jpeg(image_features['image_raw'],channels = 3)
    image = tf.cast(tf.image.resize(image,size=(416,416)), tf.uint8)
    bboxes =  tf.io.parse_tensor(image_features['bboxes'], out_type=tf.float32)
    bboxes = tf.clip_by_value(bboxes*416,1,415)
    
    num_real_boxes = image_features['num_real_boxes']
    return image,bboxes,tf.cast(num_real_boxes,tf.int64)
    
@tf.function(input_signature=[tf.TensorSpec((416,416,3), tf.uint8),tf.TensorSpec((None,4), tf.float32),tf.TensorSpec((), tf.int64)]) 
def tf_numpy_albumentations_real(image,bboxes,num_real_boxes):
    
    boxes_shape = bboxes.shape
    im_shape = image.shape

    image,bboxes,num_real_boxes = tf.numpy_function(imgaug_data_augmentation,[image,bboxes,num_real_boxes],Tout =[tf.uint8,tf.float32,tf.int64])
 
    image.set_shape(im_shape)
    bboxes.set_shape(boxes_shape)
    print("Imagen data type",image.dtype)
    print("Bboxes data type",bboxes.dtype)
    print("num_real_boxes",num_real_boxes.dtype)

    return image,bboxes,num_real_boxes

In [4]:
#USANDO TF.IMAGE MODULE
#anchors =tf.constant(np.array([[0,0,0.015,0.037],[0,0,0.043,0.104],[0,0,0.11,0.278],[0,0,0.351,0.66]]),dtype=tf.float32)
anchors =tf.constant(np.array([[0,0,0.026,0.062],[0,0,0.067,0.183],[0,0,0.128,0.323],[0,0,0.343,0.650]]),dtype=tf.float32)

os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")
filenames = os. listdir()
raw_image_dataset = tf.data.TFRecordDataset(filenames)

os.chdir("/home/sergio/Documents/pedestrian_dataset_val_tfr")
filenames = os. listdir()
raw_image_dataset_val =tf.data.TFRecordDataset(filenames)

os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")
#.shuffle(70000)
train_dataset = raw_image_dataset.map(preprocessing,num_parallel_calls=8)
train_dataset = train_dataset.map(tf_numpy_albumentations_real,num_parallel_calls=8)
train_dataset = train_dataset.map(lambda x,y,z:build_targets(x,y,z,anchors),num_parallel_calls=8)
train_dataset = train_dataset.batch(16)

val_dataset = raw_image_dataset_val.map(preprocessing_validation_set,num_parallel_calls=8)
val_dataset = val_dataset.map(lambda x,y,z:build_targets(x,y,z,anchors),num_parallel_calls=8)
val_dataset = val_dataset.batch(16)

Imagen data type <dtype: 'uint8'>
Bboxes data type <dtype: 'float32'>
num_real_boxes <dtype: 'int64'>
Tensor("ArgMax:0", shape=(None,), dtype=int64)
Tensor("strided_slice_2:0", shape=(None,), dtype=int64)
Tensor("GatherV2:0", shape=(None,), dtype=int64)
Tensor("strided_slice_3:0", shape=(None,), dtype=int64)
Tensor("GatherV2_1:0", shape=(None,), dtype=int64)
tipo de aoutput_positivon Tensor("add_18:0", shape=(None, 1), dtype=int64)
Tensor("add_18:0", shape=(None, 1), dtype=int64)
tipo de aoutput_positivon Tensor("add_30:0", shape=(None, 1), dtype=int64)
Tensor("add_30:0", shape=(None, 1), dtype=int64)
Tensor("ArgMax:0", dtype=int64)
Tensor("strided_slice_2:0", shape=(None,), dtype=int64)
Tensor("GatherV2:0", dtype=int64)
Tensor("strided_slice_3:0", shape=(None,), dtype=int64)
Tensor("GatherV2_1:0", dtype=int64)
tipo de aoutput_positivon Tensor("add_18:0", dtype=int64)
Tensor("add_18:0", dtype=int64)
tipo de aoutput_positivon Tensor("add_30:0", dtype=int64)
Tensor("add_30:0", dtype=int6

### Loss Function

In [5]:
from tensorflow.keras.losses import Loss,BinaryCrossentropy,MeanSquaredError,MeanSquaredLogarithmicError
'''
def loss_xy(y_true,y_pred):
    
    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tx_true,ty_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tx_pred,ty_pred = tf.split(y_pred, [1,1], axis=-1)
    
    loss_x = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tx_true,tx_pred)[:,:,tf.newaxis]),axis=1))
    loss_y = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(ty_true,ty_pred)[:,:,tf.newaxis]),axis=1))

    return loss_x+loss_y

def loss_wh(y_true,y_pred):
    
    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)
    
    loss_w = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tw_true,tw_pred)[:,:,tf.newaxis]),axis=1))
    loss_h = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(th_true,th_pred)[:,:,tf.newaxis]),axis=1))


    return loss_w+loss_h
'''

def loss_xy(y_true,y_pred):
    
    msle = MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.NONE)

    tx_true,ty_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tx_pred,ty_pred = tf.split(y_pred, [1,1], axis=-1)
    
    #print(tf.math.square(tx_true-tx_pred))  
    
    loss_x = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(tx_true,tx_pred)[:,:,tf.newaxis]),axis=1))
    loss_y = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(ty_true,ty_pred)[:,:,tf.newaxis]),axis=1))

    return loss_x+loss_y

def loss_wh(y_true,y_pred):

    msle = MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.NONE)
    
    tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)
    
    #print(tf.math.square(tw_true-tw_pred))

    
    loss_w = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(tw_true,tw_pred)[:,:,tf.newaxis]),axis=1))
    loss_h = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(th_true,th_pred)[:,:,tf.newaxis]),axis=1))


    return loss_w+loss_h

def loss_objectness(y_true,y_pred):
    bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    
    #tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    #tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)   
    
    loss_obj =tf.reduce_mean(tf.reduce_sum( y_true*bce(y_true,y_pred)[:,tf.newaxis],axis=1))
    
    return loss_obj

def loss_no_objectness(y_true,y_pred):
    bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    
    
    loss_noobj =tf.reduce_mean(tf.reduce_sum((1-y_true)*bce(y_true,y_pred)[:,tf.newaxis],axis=1))
    
    return loss_noobj
'''
def loss_bce_objectness(y_true,y_pred):
    #bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    #tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    #tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)   
    
    loss_obj =tf.reduce_mean(tf.reduce_sum( y_true*tf.math.log(y_pred+1e-7)[:,tf.newaxis],axis=1))
    
    return loss_obj

def loss_bce_no_objectness(y_true,y_pred):
    #bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    loss_noobj =tf.reduce_mean(tf.reduce_sum((1-y_true)*tf.math.log(1-y_pred+1e-7)[:,tf.newaxis],axis=1))
    
    return loss_noobj
'''

'\ndef loss_bce_objectness(y_true,y_pred):\n    #bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)\n    #tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)\n    #tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)   \n    \n    loss_obj =tf.reduce_mean(tf.reduce_sum( y_true*tf.math.log(y_pred+1e-7)[:,tf.newaxis],axis=1))\n    \n    return loss_obj\n\ndef loss_bce_no_objectness(y_true,y_pred):\n    #bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)\n    loss_noobj =tf.reduce_mean(tf.reduce_sum((1-y_true)*tf.math.log(1-y_pred+1e-7)[:,tf.newaxis],axis=1))\n    \n    return loss_noobj\n'

## Eager execution training (For debuging)

In [44]:
debugging = False

if debugging:
    model = TinyYOLOv3(1,anchor_boxes=anchors,train=True)
    model.build(batch_input_shape=(None,416,416,3))
    model.summary()
    model.load_weights_darknet("/home/sergio/TinyYOLOv3-Pedestrian-Detection/yolov3-tiny.weights");
    optimizer = tf.keras.optimizers.Adam()
    #model_loss = YOLOLoss()

    for epochs in range(1,2,1):
        for (images,y_true) in train_dataset:
            with tf.GradientTape() as tape:
                outputs = model(images)

                loss_x_y = loss_xy(y_true[0],outputs[0])
                loss_w_h = loss_wh(y_true[1],outputs[1])
                loss_obj = loss_objectness(y_true[2],outputs[2])
                loss_noobj = loss_no_objectness(y_true[3],outputs[3])

                total_loss =loss_x_y+loss_w_h+loss_obj+loss_noobj
            #print(epochs,(total_loss.numpy()))
            grads = tape.gradient(total_loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

                #logging.info("{}_train_{}, {}, {}".format(
                #    epoch, batch, total_loss.numpy()))
                
else:
    pass

## Training with Model.fit()

In [60]:
anchors =tf.constant(np.array([[0.026,0.062],[0.067,0.183],[0.128,0.323],[0.343,0.650]]),dtype=tf.float32)
model = TinyYOLOv3(1,anchor_boxes=anchors,train=True,mode = "finetuning")
model.build(batch_input_shape=(None,416,416,3))
print(model.load_weights_darknet("/home/sergio/TinyYOLOv3-Pedestrian-Detection/yolov3-tiny.weights"))

Pesos de la convolucion (432,)
CONV SHAPE (16, 3, 3, 3)
Pesos de la convolucion (4608,)
CONV SHAPE (32, 16, 3, 3)
Pesos de la convolucion (18432,)
CONV SHAPE (64, 32, 3, 3)
Pesos de la convolucion (73728,)
CONV SHAPE (128, 64, 3, 3)
Pesos de la convolucion (294912,)
CONV SHAPE (256, 128, 3, 3)
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
Pesos de la convolucion (4718592,)
CONV SHAPE (1024, 512, 3, 3)
Pesos de la convolucion (262144,)
CONV SHAPE (256, 1024, 1, 1)
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
Pesos de la convolucion (130560,)
CONV SHAPE (255, 512, 1, 1)
Pesos de la convolucion (32768,)
CONV SHAPE (128, 256, 1, 1)
Pesos de la convolucion (884736,)
CONV SHAPE (256, 384, 3, 3)
Pesos de la convolucion (65280,)
CONV SHAPE (255, 256, 1, 1)
8858734


In [61]:
for l in model.layers:
    print(l.name, l.trainable)
    pass

BasicBlock1 True
BasicBlock2 True
BasicBlock3 True
BasicBlock4 True
BasicBlock5 True
BasicBlock6 True
BasicBlock7 True
BasicBlock8 True
BasicBlock9 True
FinalBlock1 True
BasicBlock11 True
BasicBlock12 True
FinalBlock2 True
Concatenate True
Upsampling True
Prediction1 True
Prediction2 True
Concatenate_BBOX True


In [62]:
tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Nadam(learning_rate=0.001)

losses = {"output_1": loss_xy,
          "output_2": loss_wh,
          "output_3":loss_objectness,
          "output_4":loss_no_objectness  
}

metrics = {"output_3":[Precision(),Recall(),TrueNegatives(),TruePositives(),FalseNegatives(),FalsePositives()]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[1,2,4,1])
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")

## Batch 16, con pesos 1,2,4,1 with lr=0.001 y usando otro loss para xy e wh

In [63]:
history = model.fit(train_dataset, epochs=10,validation_data=val_dataset)

Epoch 1/10
4607/4607 [==============================] - 533s 116ms/step - loss: 16.9286 - output_1_loss: 0.0016 - output_2_loss: 0.1968 - output_3_loss: 0.0620 - output_4_loss: 16.2855 - output_3_precision_9: 0.2960 - output_3_recall_9: 0.0836 - output_3_true_negatives_9: 124226912.0000 - output_3_true_positives_9: 22508.0000 - output_3_false_negatives_9: 246587.0000 - output_3_false_positives_9: 53542.0000 - val_loss: 13.4692 - val_output_1_loss: 3.3242e-07 - val_output_2_loss: 0.1618 - val_output_3_loss: 0.0576 - val_output_4_loss: 12.9151 - val_output_3_precision_9: 0.6865 - val_output_3_recall_9: 0.1266 - val_output_3_true_negatives_9: 5057539.0000 - val_output_3_true_positives_9: 1491.0000 - val_output_3_false_negatives_9: 10289.0000 - val_output_3_false_positives_9: 681.0000
Epoch 2/10
4607/4607 [==============================] - 547s 119ms/step - loss: 13.2240 - output_1_loss: 4.3912e-06 - output_2_loss: 0.1608 - output_3_loss: 0.0551 - output_4_loss: 12.6819 - output_3_precisio

In [64]:
os.chdir("/home/sergio/Documents/json_experiments")
import json
#json.dumps(str(a))
with open('history_finetuning_1241_10_epoch_nadam_0dot001_msle.json', 'w') as fp:
    json.dump(str(history.history), fp)

In [65]:
model.save_weights('/home/sergio/Documents/weights_saved/pesos_10_epochs_nadam_1_2_4_1_0dot001_msle')

###  Continuacion MSLE con lr=0.001, 50 epocas

In [6]:
anchors =tf.constant(np.array([[0.026,0.062],[0.067,0.183],[0.128,0.323],[0.343,0.650]]),dtype=tf.float32)
model = TinyYOLOv3(1,anchor_boxes=anchors,train=True,mode = "finetuning")
model.build(batch_input_shape=(None,416,416,3))
print(model.load_weights_darknet("/home/sergio/TinyYOLOv3-Pedestrian-Detection/yolov3-tiny.weights"))
model.load_weights('/home/sergio/Documents/weights_saved/pesos_10_epochs_nadam_1_2_4_1_0dot001_msle')

Pesos de la convolucion (432,)
CONV SHAPE (16, 3, 3, 3)
Pesos de la convolucion (4608,)
CONV SHAPE (32, 16, 3, 3)
Pesos de la convolucion (18432,)
CONV SHAPE (64, 32, 3, 3)
Pesos de la convolucion (73728,)
CONV SHAPE (128, 64, 3, 3)
Pesos de la convolucion (294912,)
CONV SHAPE (256, 128, 3, 3)
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
Pesos de la convolucion (4718592,)
CONV SHAPE (1024, 512, 3, 3)
Pesos de la convolucion (262144,)
CONV SHAPE (256, 1024, 1, 1)
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
Pesos de la convolucion (130560,)
CONV SHAPE (255, 512, 1, 1)
Pesos de la convolucion (32768,)
CONV SHAPE (128, 256, 1, 1)
Pesos de la convolucion (884736,)
CONV SHAPE (256, 384, 3, 3)
Pesos de la convolucion (65280,)
CONV SHAPE (255, 256, 1, 1)
8858734


In [7]:
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")

In [8]:
for l in model.layers:
    print(l.name, l.trainable)
    pass

BasicBlock1 True
BasicBlock2 True
BasicBlock3 True
BasicBlock4 True
BasicBlock5 True
BasicBlock6 True
BasicBlock7 True
BasicBlock8 True
BasicBlock9 True
FinalBlock1 True
BasicBlock11 True
BasicBlock12 True
FinalBlock2 True
Concatenate True
Upsampling True
Prediction1 True
Prediction2 True
Concatenate_BBOX True


In [9]:
tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Nadam(learning_rate=0.001)

losses = {"output_1": loss_xy,
          "output_2": loss_wh,
          "output_3":loss_objectness,
          "output_4":loss_no_objectness  
}

metrics = {"output_3":[Precision(),Recall(),TrueNegatives(),TruePositives(),FalseNegatives(),FalsePositives()]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[1,2,4,1])
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")

In [10]:
history2 = model.fit(train_dataset, epochs=50,validation_data=val_dataset)

Epoch 1/50
4607/4607 [==============================] - 529s 115ms/step - loss: 10.1232 - output_1_loss: 8.4822e-07 - output_2_loss: 0.1229 - output_3_loss: 0.0433 - output_4_loss: 9.7040 - output_3_precision: 0.7211 - output_3_recall: 0.2911 - output_3_true_negatives: 124250248.0000 - output_3_true_positives: 78323.0000 - output_3_false_negatives: 190754.0000 - output_3_false_positives: 30299.0000 - val_loss: 9.2768 - val_output_1_loss: 0.0000e+00 - val_output_2_loss: 0.1251 - val_output_3_loss: 0.0411 - val_output_4_loss: 8.8623 - val_output_3_precision: 0.7807 - val_output_3_recall: 0.3490 - val_output_3_true_negatives: 5057065.0000 - val_output_3_true_positives: 4111.0000 - val_output_3_false_negatives: 7669.0000 - val_output_3_false_positives: 1155.0000
Epoch 2/50
4607/4607 [==============================] - 536s 116ms/step - loss: 9.8998 - output_1_loss: 2.0321e-06 - output_2_loss: 0.1223 - output_3_loss: 0.0425 - output_4_loss: 9.4850 - output_3_precision: 0.7269 - output_3_reca

In [11]:
os.chdir("/home/sergio/Documents/json_experiments")
import json
#json.dumps(str(a))
with open('history_finetuning_1241_10_60_epoch_nadam_0dot001_msle.json', 'w') as fp:
    json.dump(str(history2.history), fp)

In [12]:
model.save_weights('/home/sergio/Documents/weights_saved/pesos_10_60_epochs_nadam_1_2_4_1_0dot001_msle')

In [13]:
tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Nadam(learning_rate=0.0001)

losses = {"output_1": loss_xy,
          "output_2": loss_wh,
          "output_3":loss_objectness,
          "output_4":loss_no_objectness  
}

metrics = {"output_3":[Precision(),Recall(),TrueNegatives(),TruePositives(),FalseNegatives(),FalsePositives()]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[1,2,4,1])
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")

In [14]:
history3 = model.fit(train_dataset, epochs=20,validation_data=val_dataset)

Epoch 1/20
4607/4607 [==============================] - 565s 123ms/step - loss: 5.4410 - output_1_loss: 1.8831e-05 - output_2_loss: 0.0659 - output_3_loss: 0.0238 - output_4_loss: 5.2140 - output_3_precision_1: 0.8820 - output_3_recall_1: 0.6528 - output_3_true_negatives_1: 124257016.0000 - output_3_true_positives_1: 175643.0000 - output_3_false_negatives_1: 93438.0000 - output_3_false_positives_1: 23502.0000 - val_loss: 1.6228 - val_output_1_loss: 0.0000e+00 - val_output_2_loss: 0.0605 - val_output_3_loss: 0.0075 - val_output_4_loss: 1.4719 - val_output_3_precision_1: 0.9665 - val_output_3_recall_1: 0.9229 - val_output_3_true_negatives_1: 5057843.0000 - val_output_3_true_positives_1: 10872.0000 - val_output_3_false_negatives_1: 908.0000 - val_output_3_false_positives_1: 377.0000
Epoch 2/20
4607/4607 [==============================] - 546s 119ms/step - loss: 5.3190 - output_1_loss: 6.4858e-07 - output_2_loss: 0.0645 - output_3_loss: 0.0233 - output_4_loss: 5.0969 - output_3_precision_1

In [15]:
os.chdir("/home/sergio/Documents/json_experiments")
import json
#json.dumps(str(a))
with open('history_finetuning_1241_60_80_epoch_nadam_0dot0001_msle.json', 'w') as fp:
    json.dump(str(history3.history), fp)

In [16]:
model.save_weights('/home/sergio/Documents/weights_saved/pesos_60_80_epochs_nadam_1_2_4_1_0dot0001_msle')

## Continuación de entrenamiento con msle moviendo los pesos a 1441, con lr 0.0001

In [6]:
anchors =tf.constant(np.array([[0.026,0.062],[0.067,0.183],[0.128,0.323],[0.343,0.650]]),dtype=tf.float32)
model = TinyYOLOv3(1,anchor_boxes=anchors,train=True,mode = "finetuning")
model.build(batch_input_shape=(None,416,416,3))
print(model.load_weights_darknet("/home/sergio/TinyYOLOv3-Pedestrian-Detection/yolov3-tiny.weights"))
model.load_weights('/home/sergio/Documents/weights_saved/pesos_60_80_epochs_nadam_1_2_4_1_0dot0001_msle')

Pesos de la convolucion (432,)
CONV SHAPE (16, 3, 3, 3)
Pesos de la convolucion (4608,)
CONV SHAPE (32, 16, 3, 3)
Pesos de la convolucion (18432,)
CONV SHAPE (64, 32, 3, 3)
Pesos de la convolucion (73728,)
CONV SHAPE (128, 64, 3, 3)
Pesos de la convolucion (294912,)
CONV SHAPE (256, 128, 3, 3)
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
Pesos de la convolucion (4718592,)
CONV SHAPE (1024, 512, 3, 3)
Pesos de la convolucion (262144,)
CONV SHAPE (256, 1024, 1, 1)
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
Pesos de la convolucion (130560,)
CONV SHAPE (255, 512, 1, 1)
Pesos de la convolucion (32768,)
CONV SHAPE (128, 256, 1, 1)
Pesos de la convolucion (884736,)
CONV SHAPE (256, 384, 3, 3)
Pesos de la convolucion (65280,)
CONV SHAPE (255, 256, 1, 1)
8858734


In [7]:
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")

In [8]:
for l in model.layers:
    print(l.name, l.trainable)
    pass

BasicBlock1 True
BasicBlock2 True
BasicBlock3 True
BasicBlock4 True
BasicBlock5 True
BasicBlock6 True
BasicBlock7 True
BasicBlock8 True
BasicBlock9 True
FinalBlock1 True
BasicBlock11 True
BasicBlock12 True
FinalBlock2 True
Concatenate True
Upsampling True
Prediction1 True
Prediction2 True
Concatenate_BBOX True


In [9]:
tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Nadam(learning_rate=0.0001)

losses = {"output_1": loss_xy,
          "output_2": loss_wh,
          "output_3":loss_objectness,
          "output_4":loss_no_objectness  
}

metrics = {"output_3":[Precision(),Recall(),TrueNegatives(),TruePositives(),FalseNegatives(),FalsePositives()]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[1,4,4,1])
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")

In [10]:
history4 = model.fit(train_dataset, epochs=20,validation_data=val_dataset)

Epoch 1/20
4607/4607 [==============================] - 541s 118ms/step - loss: 4.9830 - output_1_loss: 2.3880e-07 - output_2_loss: 0.0623 - output_3_loss: 0.0213 - output_4_loss: 4.6486 - output_3_precision: 0.9059 - output_3_recall: 0.6939 - output_3_true_negatives: 124261088.0000 - output_3_true_positives: 186691.0000 - output_3_false_negatives: 82348.0000 - output_3_false_positives: 19384.0000 - val_loss: 1.0454 - val_output_1_loss: 0.0000e+00 - val_output_2_loss: 0.0556 - val_output_3_loss: 0.0041 - val_output_4_loss: 0.8065 - val_output_3_precision: 0.9866 - val_output_3_recall: 0.9771 - val_output_3_true_negatives: 5058064.0000 - val_output_3_true_positives: 11510.0000 - val_output_3_false_negatives: 270.0000 - val_output_3_false_positives: 156.0000
Epoch 2/20
4607/4607 [==============================] - 542s 118ms/step - loss: 4.9659 - output_1_loss: 9.3114e-07 - output_2_loss: 0.0620 - output_3_loss: 0.0214 - output_4_loss: 4.6325 - output_3_precision: 0.9060 - output_3_recall

In [13]:
os.chdir("/home/sergio/Documents/json_experiments")
import json
#json.dumps(str(a))
with open('history_finetuning_1441_80_100_epoch_nadam_0dot0001_msle.json', 'w') as fp:
    json.dump(str(history4.history), fp)

In [14]:
model.save_weights('/home/sergio/Documents/weights_saved/pesos_80_100_epochs_nadam_1_4_4_1_0dot0001_msle')

In [15]:
tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Nadam(learning_rate=0.00001)

losses = {"output_1": loss_xy,
          "output_2": loss_wh,
          "output_3":loss_objectness,
          "output_4":loss_no_objectness  
}

metrics = {"output_3":[Precision(),Recall(),TrueNegatives(),TruePositives(),FalseNegatives(),FalsePositives()]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[1,4,4,1])
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")

In [16]:
history5 = model.fit(train_dataset, epochs=20,validation_data=val_dataset)

Epoch 1/20
4607/4607 [==============================] - 514s 111ms/step - loss: 4.8021 - output_1_loss: 1.3133e-05 - output_2_loss: 0.0589 - output_3_loss: 0.0206 - output_4_loss: 4.4838 - output_3_precision_1: 0.9096 - output_3_recall_1: 0.7023 - output_3_true_negatives_1: 124261792.0000 - output_3_true_positives_1: 188967.0000 - output_3_false_negatives_1: 80108.0000 - output_3_false_positives_1: 18773.0000 - val_loss: 0.9312 - val_output_1_loss: 0.0000e+00 - val_output_2_loss: 0.0527 - val_output_3_loss: 0.0035 - val_output_4_loss: 0.7062 - val_output_3_precision_1: 0.9917 - val_output_3_recall_1: 0.9820 - val_output_3_true_negatives_1: 5058123.0000 - val_output_3_true_positives_1: 11568.0000 - val_output_3_false_negatives_1: 212.0000 - val_output_3_false_positives_1: 97.0000
Epoch 2/20
4607/4607 [==============================] - 515s 112ms/step - loss: 4.7625 - output_1_loss: 0.0000e+00 - output_2_loss: 0.0586 - output_3_loss: 0.0204 - output_4_loss: 4.4464 - output_3_precision_1:

In [17]:
os.chdir("/home/sergio/Documents/json_experiments")
import json
#json.dumps(str(a))
with open('history_finetuning_1441_100_120_epoch_nadam_0dot00001_msle.json', 'w') as fp:
    json.dump(str(history4.history), fp)

In [18]:
model.save_weights('/home/sergio/Documents/weights_saved/pesos_100_120_epochs_nadam_1_4_4_1_0dot00001_msle')

## Batch 16, con pesos 2,1,4,1 con lr 0.001 ESTE ES POSIBLEMENTE EL BUENO

In [7]:
anchors =tf.constant(np.array([[0,0,0.026,0.062],[0,0,0.067,0.183],[0,0,0.128,0.323],[0,0,0.343,0.650]]),dtype=tf.float32)
model = TinyYOLOv3(1,anchor_boxes=anchors,train=True,mode = "finetuning")
model.build(batch_input_shape=(None,416,416,3))
print(model.load_weights_darknet("/home/sergio/TinyYOLOv3-Pedestrian-Detection/yolov3-tiny.weights"))

Pesos de la convolucion (432,)
CONV SHAPE (16, 3, 3, 3)
Pesos de la convolucion (4608,)
CONV SHAPE (32, 16, 3, 3)
Pesos de la convolucion (18432,)
CONV SHAPE (64, 32, 3, 3)
Pesos de la convolucion (73728,)
CONV SHAPE (128, 64, 3, 3)
Pesos de la convolucion (294912,)
CONV SHAPE (256, 128, 3, 3)
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
Pesos de la convolucion (4718592,)
CONV SHAPE (1024, 512, 3, 3)
Pesos de la convolucion (262144,)
CONV SHAPE (256, 1024, 1, 1)
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
Pesos de la convolucion (130560,)
CONV SHAPE (255, 512, 1, 1)
Pesos de la convolucion (32768,)
CONV SHAPE (128, 256, 1, 1)
Pesos de la convolucion (884736,)
CONV SHAPE (256, 384, 3, 3)
Pesos de la convolucion (65280,)
CONV SHAPE (255, 256, 1, 1)
8858734


In [8]:
tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Nadam(learning_rate=0.001)

losses = {"output_1": loss_xy,
          "output_2": loss_wh,
          "output_3":loss_objectness,
          "output_4":loss_no_objectness  
}

metrics = {"output_3":[Precision(),Recall(),TrueNegatives(),TruePositives(),FalseNegatives(),FalsePositives()]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[2,1,4,1])
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")

In [10]:
history = model.fit(train_dataset, epochs=50,validation_data=val_dataset)

Epoch 1/50
4607/4607 [==============================] - 549s 119ms/step - loss: 34.3996 - output_1_loss: 6.1519 - output_2_loss: 2.7798 - output_3_loss: 0.0669 - output_4_loss: 19.0485 - output_3_precision: 0.1138 - output_3_recall: 0.0564 - output_3_true_negatives: 124162384.0000 - output_3_true_positives: 15180.0000 - output_3_false_negatives: 253944.0000 - output_3_false_positives: 118220.0000 - val_loss: 27.0672 - val_output_1_loss: 5.6957 - val_output_2_loss: 1.3486 - val_output_3_loss: 0.0626 - val_output_4_loss: 14.0769 - val_output_3_precision: 0.6688 - val_output_3_recall: 0.0789 - val_output_3_true_negatives: 5057760.0000 - val_output_3_true_positives: 929.0000 - val_output_3_false_negatives: 10851.0000 - val_output_3_false_positives: 460.0000
Epoch 2/50
4607/4607 [==============================] - 547s 119ms/step - loss: 26.7667 - output_1_loss: 5.4479 - output_2_loss: 2.3739 - output_3_loss: 0.0574 - output_4_loss: 13.2674 - output_3_precision: 0.6450 - output_3_recall: 0.0

In [13]:
os.chdir("/home/sergio/Documents/json_experiments")
import json
#json.dumps(str(a))
with open('history_finetuning_2141_50_epoch_nadam_0dot001.json', 'w') as fp:
    json.dump(str(history.history), fp)

In [14]:
model.save_weights('/home/sergio/Documents/weights_saved/pesos_50_epochs_nadam_2_1_4_1_0dot001_el_bueno')

In [15]:
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")

In [16]:
history_part2 = model.fit(train_dataset, epochs=30,validation_data=val_dataset)

Epoch 1/30
4607/4607 [==============================] - 509s 110ms/step - loss: 14.6874 - output_1_loss: 2.7555 - output_2_loss: 1.3629 - output_3_loss: 0.0352 - output_4_loss: 7.6727 - output_3_precision: 0.7940 - output_3_recall: 0.4551 - output_3_true_negatives: 124248656.0000 - output_3_true_positives: 122467.0000 - output_3_false_negatives: 146625.0000 - output_3_false_positives: 31777.0000 - val_loss: 8.7461 - val_output_1_loss: 1.3432 - val_output_2_loss: 0.5087 - val_output_3_loss: 0.0269 - val_output_4_loss: 5.4434 - val_output_3_precision: 0.8510 - val_output_3_recall: 0.6226 - val_output_3_true_negatives: 5056936.0000 - val_output_3_true_positives: 7334.0000 - val_output_3_false_negatives: 4446.0000 - val_output_3_false_positives: 1284.0000
Epoch 2/30
4607/4607 [==============================] - 528s 115ms/step - loss: 14.6643 - output_1_loss: 2.7374 - output_2_loss: 1.4411 - output_3_loss: 0.0349 - output_4_loss: 7.6090 - output_3_precision: 0.7955 - output_3_recall: 0.4589

In [18]:
os.chdir("/home/sergio/Documents/json_experiments")
import json
#json.dumps(str(a))
with open('history_finetuning_2141_50_to_80_epoch_nadam_0dot001.json', 'w') as fp:
    json.dump(str(history_part2.history), fp)

In [19]:
model.save_weights('/home/sergio/Documents/weights_saved/pesos_50_to_80_epochs_nadam_2_1_4_1_0dot001_el_bueno')

In [20]:
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")

In [21]:
history_part3 = model.fit(train_dataset, epochs=10,validation_data=val_dataset)

Epoch 1/10
4607/4607 [==============================] - 522s 113ms/step - loss: 13.0215 - output_1_loss: 2.5013 - output_2_loss: 1.3627 - output_3_loss: 0.0301 - output_4_loss: 6.5360 - output_3_precision: 0.8338 - output_3_recall: 0.5523 - output_3_true_negatives: 124251008.0000 - output_3_true_positives: 148666.0000 - output_3_false_negatives: 120500.0000 - output_3_false_positives: 29632.0000 - val_loss: 6.3861 - val_output_1_loss: 1.0648 - val_output_2_loss: 0.4577 - val_output_3_loss: 0.0187 - val_output_4_loss: 3.7241 - val_output_3_precision: 0.8922 - val_output_3_recall: 0.7635 - val_output_3_true_negatives: 5057133.0000 - val_output_3_true_positives: 8994.0000 - val_output_3_false_negatives: 2786.0000 - val_output_3_false_positives: 1087.0000 511s 112ms/step - loss: 13.0014 - output_1_loss: 2.4981 - output_2_loss: 1.3597 - output_3_loss: 0.0300 - output_4_loss: 6.5255 - output_3_precision: 0.8339 - output_3_recall: 0.5526 - output_3_true_negatives: 122683408.00
Epoch 2/10
4607

In [22]:
os.chdir("/home/sergio/Documents/json_experiments")
import json
#json.dumps(str(a))
with open('history_finetuning_2141_80_to_90_epoch_nadam_0dot001.json', 'w') as fp:
    json.dump(str(history_part3.history), fp)

In [23]:
model.save_weights('/home/sergio/Documents/weights_saved/pesos_80_to_90_epochs_nadam_2_1_4_1_0dot001_el_bueno')

In [6]:
anchors =[[0.015,0.037],[0.043,0.104],[0.11,0.278],[0.351,0.66]]
model = TinyYOLOv3(1,anchor_boxes=anchors,train=True,mode = "finetuning")
model.build(batch_input_shape=(None,416,416,3))
print(model.load_weights_darknet("/home/sergio/TinyYOLOv3-Pedestrian-Detection/yolov3-tiny.weights"))
model.load_weights('/home/sergio/Documents/weights_saved/pesos_80_to_90_epochs_nadam_2_1_4_1_0dot001_el_bueno')

Pesos de la convolucion (432,)
CONV SHAPE (16, 3, 3, 3)
Pesos de la convolucion (4608,)
CONV SHAPE (32, 16, 3, 3)
Pesos de la convolucion (18432,)
CONV SHAPE (64, 32, 3, 3)
Pesos de la convolucion (73728,)
CONV SHAPE (128, 64, 3, 3)
Pesos de la convolucion (294912,)
CONV SHAPE (256, 128, 3, 3)
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
Pesos de la convolucion (4718592,)
CONV SHAPE (1024, 512, 3, 3)
Pesos de la convolucion (262144,)
CONV SHAPE (256, 1024, 1, 1)
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
Pesos de la convolucion (130560,)
CONV SHAPE (255, 512, 1, 1)
Pesos de la convolucion (32768,)
CONV SHAPE (128, 256, 1, 1)
Pesos de la convolucion (884736,)
CONV SHAPE (256, 384, 3, 3)
Pesos de la convolucion (65280,)
CONV SHAPE (255, 256, 1, 1)
8858734


In [7]:
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")

In [8]:
for l in model.layers:
    print(l.name, l.trainable)
    pass

BasicBlock1 True
BasicBlock2 True
BasicBlock3 True
BasicBlock4 True
BasicBlock5 True
BasicBlock6 True
BasicBlock7 True
BasicBlock8 True
BasicBlock9 True
FinalBlock1 True
BasicBlock11 True
BasicBlock12 True
FinalBlock2 True
Concatenate True
Upsampling True
Prediction1 True
Prediction2 True
Concatenate_BBOX True


In [9]:
tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Nadam(learning_rate=0.001)

losses = {"output_1": loss_xy,
          "output_2": loss_wh,
          "output_3":loss_objectness,
          "output_4":loss_no_objectness  
}

metrics = {"output_3":[Precision(),Recall(),TrueNegatives(),TruePositives(),FalseNegatives(),FalsePositives()]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[2,1,4,1])
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")

In [10]:
history_part4 = model.fit(train_dataset, epochs=50,validation_data=val_dataset)

Epoch 1/50
4607/4607 [==============================] - 533s 116ms/step - loss: 12.5533 - output_1_loss: 2.4457 - output_2_loss: 1.2728 - output_3_loss: 0.0288 - output_4_loss: 6.2741 - output_3_precision: 0.8434 - output_3_recall: 0.5742 - output_3_true_negatives: 124251872.0000 - output_3_true_positives: 154503.0000 - output_3_false_negatives: 114565.0000 - output_3_false_positives: 28687.0000 - val_loss: 5.6376 - val_output_1_loss: 0.9970 - val_output_2_loss: 0.4249 - val_output_3_loss: 0.0158 - val_output_4_loss: 3.1556 - val_output_3_precision: 0.9072 - val_output_3_recall: 0.8126 - val_output_3_true_negatives: 5057241.0000 - val_output_3_true_positives: 9572.0000 - val_output_3_false_negatives: 2208.0000 - val_output_3_false_positives: 979.0000
Epoch 2/50
4607/4607 [==============================] - 532s 115ms/step - loss: 12.5383 - output_1_loss: 2.4540 - output_2_loss: 1.2919 - output_3_loss: 0.0285 - output_4_loss: 6.2243 - output_3_precision: 0.8451 - output_3_recall: 0.5787 

In [13]:
(2*0.9424*0.8701)/(0.9424+0.8701)

0.9048079889655172

In [14]:
(2*0.9372*0.8777)/(0.9372+0.8777)

0.9064746707807593

In [11]:
os.chdir("/home/sergio/Documents/json_experiments")
import json
#json.dumps(str(a))
with open('history_finetuning_2141_90_to_140_epoch_nadam_0dot001.json', 'w') as fp:
    json.dump(str(history_part4.history), fp)

In [12]:
model.save_weights('/home/sergio/Documents/weights_saved/pesos_90_to_140_epochs_nadam_2_1_4_1_0dot001_el_bueno')

In [15]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/home/sergio/Documents/best_weights_saved',
    save_weights_only=True,
    monitor='val_output_3_recall',
    mode='max',
    save_best_only=True)

In [17]:
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")
history_part5 = model.fit(train_dataset, epochs=20,validation_data=val_dataset,callbacks=[model_checkpoint_callback])

Epoch 1/20
4607/4607 [==============================] - 550s 119ms/step - loss: 11.4285 - output_1_loss: 2.3045 - output_2_loss: 1.1677 - output_3_loss: 0.0254 - output_4_loss: 5.5505 - output_3_precision: 0.8719 - output_3_recall: 0.6337 - output_3_true_negatives: 124255568.0000 - output_3_true_positives: 170498.0000 - output_3_false_negatives: 98536.0000 - output_3_false_positives: 25051.0000 - val_loss: 4.3978 - val_output_1_loss: 0.8296 - val_output_2_loss: 0.4226 - val_output_3_loss: 0.0109 - val_output_4_loss: 2.2724 - val_output_3_precision: 0.9320 - val_output_3_recall: 0.8781 - val_output_3_true_negatives: 5057465.0000 - val_output_3_true_positives: 10344.0000 - val_output_3_false_negatives: 1436.0000 - val_output_3_false_positives: 755.0000
Epoch 2/20
4607/4607 [==============================] - 538s 117ms/step - loss: 11.5216 - output_1_loss: 2.3269 - output_2_loss: 1.1737 - output_3_loss: 0.0256 - output_4_loss: 5.5916 - output_3_precision: 0.8704 - output_3_recall: 0.6313 

In [18]:
os.chdir("/home/sergio/Documents/json_experiments")
import json
#json.dumps(str(a))
with open('history_finetuning_2141_140_to_160_epoch_nadam_0dot001.json', 'w') as fp:
    json.dump(str(history_part5.history), fp)

In [19]:
model.save_weights('/home/sergio/Documents/weights_saved/pesos_140_to_160_epochs_nadam_2_1_4_1_0dot001_el_bueno')

### Experimento disminuyendo el learning rate por 10 osea 0.00001

In [20]:
tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Nadam(learning_rate=0.0001)

losses = {"output_1": loss_xy,
          "output_2": loss_wh,
          "output_3":loss_objectness,
          "output_4":loss_no_objectness  
}

metrics = {"output_3":[Precision(),Recall(),TrueNegatives(),TruePositives(),FalseNegatives(),FalsePositives()]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[2,1,4,1])
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")

In [21]:
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")
history_part5 = model.fit(train_dataset, epochs=20,validation_data=val_dataset)

Epoch 1/20
4607/4607 [==============================] - 505s 110ms/step - loss: 10.7274 - output_1_loss: 2.1723 - output_2_loss: 1.1206 - output_3_loss: 0.0236 - output_4_loss: 5.1680 - output_3_precision_1: 0.8876 - output_3_recall_1: 0.6616 - output_3_true_negatives_1: 124257792.0000 - output_3_true_positives_1: 178110.0000 - output_3_false_negatives_1: 91088.0000 - output_3_false_positives_1: 22563.0000 - val_loss: 2.9575 - val_output_1_loss: 0.5964 - val_output_2_loss: 0.3481 - val_output_3_loss: 0.0069 - val_output_4_loss: 1.3889 - val_output_3_precision_1: 0.9694 - val_output_3_recall_1: 0.9358 - val_output_3_true_negatives_1: 5057872.0000 - val_output_3_true_positives_1: 11024.0000 - val_output_3_false_negatives_1: 756.0000 - val_output_3_false_positives_1: 348.0000
Epoch 2/20
4607/4607 [==============================] - 510s 111ms/step - loss: 10.4405 - output_1_loss: 2.1064 - output_2_loss: 1.0877 - output_3_loss: 0.0230 - output_4_loss: 5.0479 - output_3_precision_1: 0.8932 -

In [22]:
os.chdir("/home/sergio/Documents/json_experiments")
import json
#json.dumps(str(a))
with open('history_finetuning_2141_160_to_180_epoch_nadam_0dot0001.json', 'w') as fp:
    json.dump(str(history_part5.history), fp)

In [23]:
model.save_weights('/home/sergio/Documents/weights_saved/pesos_160_to_180_epochs_nadam_2_1_4_1_0dot0001_el_bueno')

## CON DATA AUGMENTATION ERRONEO

In [80]:
history = model.fit(train_dataset, epochs=10,validation_data=val_dataset)

Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Epoch 1/10
Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
   9213/Unknown - 538s 58ms/step - loss: 43.1109 - output_1_loss: 6.2505 - output_2_loss: 2.3600 - output_3_loss: 0.2201 - output_4_loss: 27.3695 - output_3_precision_4: 0.4815 - output_3_recall_4: 0.0769 - output_3_true_negatives_3: 123971768.0000 - output_3_true_positives_3: 41017.0000 - output_3_false_negatives_3: 492613.0000 - output_3_false_positives_3: 44173.0000Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
9213/9213 [==============================] - 547s 59ms/step - loss: 43.1109 - output_1_loss: 6.2505 - output_2_loss: 2.3600 - output_3_loss: 0.2201 - output_4_loss: 27.3695 - output_

In [12]:
history_finetuning_4_0dot5_20_epoch_rmsprop_0dot001 = history.history
print(history_finetuning_4_0dot5_20_epoch_rmsprop_0dot001)

{'loss': [1730360.2240532888, 570.973998115753, 12400.711475089698, 45.28717758845188, 23.66998835732427, 22.33506023619514, 21.47033319496818, 21.10717370287593, 20.694593455213106, 20.383880133182274, 20.186747138893406, 19.956230268571822, 19.701506302526173, 19.54467439458679, 19.544134786408573, 19.464470668694275, 19.211783719388254, 19.194032366212575, 19.06527896271512, 18.987476273512637], 'output_1_loss': [580977.9, 162.7661, 6504.514, 6.6845803, 6.0172386, 5.834253, 5.665129, 5.6135855, 5.5386586, 5.4822116, 5.4661098, 5.3750067, 5.350316, 5.3225904, 5.326665, 5.2719526, 5.220171, 5.2109375, 5.200909, 5.1678624], 'output_2_loss': [1149076.4, 326.75598, 5845.039, 3.1532123, 2.3503883, 2.2387094, 2.0652702, 2.1025445, 2.0269985, 1.983707, 1.9550353, 1.9173115, 1.8504103, 1.8376346, 1.8716666, 1.9434427, 1.8292828, 1.8468119, 1.804593, 1.8206726], 'output_3_loss': [1.3264735, 1.1011581, 0.69467604, 0.5170728, 0.23785977, 0.22278126, 0.2154919, 0.2102615, 0.20636734, 0.20310973,

In [13]:
model_save_path = "/home/sergio/Documents/models_saved/history_finetuning_4_0dot5_20_epoch_rmsprop_0dot001"

tf.keras.models.save_model(model, model_save_path, include_optimizer=True)

Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /home/sergio/Documents/models_saved/history_finetuning_4_0dot5_20_epoch_rmsprop_0dot001/assets


In [14]:
os.chdir("/home/sergio/Documents/Checkpoints_experiements")
import json
#json.dumps(str(a))
with open('history_finetuning_4_0dot5_20_epoch_rmsprop_0dot001.json', 'w') as fp:
    json.dump(str(history_finetuning_4_0dot5_20_epoch_rmsprop_0dot001), fp)

In [15]:
for i in model.layers:
    print(i,i.trainable)

<YOLOblocks.BasicBlock object at 0x7f6f41beacf8> True
<YOLOblocks.BasicBlock object at 0x7f6f4020cbe0> True
<YOLOblocks.BasicBlock object at 0x7f6f41b4d780> True
<YOLOblocks.BasicBlock object at 0x7f6f41b4dda0> True
<YOLOblocks.BasicBlock object at 0x7f6f40180320> True
<YOLOblocks.BasicBlock object at 0x7f6f40180908> True
<YOLOblocks.BasicBlock object at 0x7f6f41b90198> True
<YOLOblocks.BasicBlock object at 0x7f6f41b90710> True
<YOLOblocks.BasicBlock object at 0x7f6f41b90c88> True
<YOLOblocks.BasicBlock object at 0x7f6f400fa240> True
<YOLOblocks.BasicBlock object at 0x7f6f400fa5c0> True
<YOLOblocks.BasicBlock object at 0x7f6f400fab38> True
<YOLOblocks.BasicBlock object at 0x7f6f401160f0> True
<tensorflow.python.keras.layers.merge.Concatenate object at 0x7f6f40116470> True
<tensorflow.python.keras.layers.convolutional.UpSampling2D object at 0x7f6f40116588> True
<YOLOblocks.PredictionLayer object at 0x7f6f401166d8> True
<YOLOblocks.PredictionLayer object at 0x7f6f40138400> True
<tensorfl

## Restart the training

In [6]:
500000*100/(1690*70000)

0.42265426880811496

In [7]:
500000*100/(1690*4700)

6.2948508120357545

## Saving a model

In [10]:
import sys
sys.path.append("/home/sergio/TinyYOLOv3-Pedestrian-Detection")

from YOLOblocks import TinyYOLOv3#,YOLOLoss
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
#from tensorflow.python.tools import freeze_graph
#from skimage.io import imread,imshow
#from skimage.transform import resize 
import time
#from tensorflow.compat.v1.image import decode_image
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import os

In [11]:
from tensorflow.keras.losses import Loss,BinaryCrossentropy,MeanSquaredError

def loss_xy(y_true,y_pred):
    
    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tx_true,ty_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tx_pred,ty_pred = tf.split(y_pred, [1,1], axis=-1)
    
    loss_x = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tx_true,tx_pred)[:,:,tf.newaxis]),axis=1))
    loss_y = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(ty_true,ty_pred)[:,:,tf.newaxis]),axis=1))

    return loss_x+loss_y

def loss_wh(y_true,y_pred):
    
    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)
    
    loss_w = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tw_true,tw_pred)[:,:,tf.newaxis]),axis=1))
    loss_h = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(th_true,th_pred)[:,:,tf.newaxis]),axis=1))


    return loss_w+loss_h

def loss_objectness(y_true,y_pred):
    bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    
    #tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    #tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)   
    
    loss_obj =tf.reduce_mean(tf.reduce_sum( y_true*bce(y_true,y_pred)[:,tf.newaxis],axis=1))
    
    return loss_obj

def loss_no_objectness(y_true,y_pred):
    bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    
    
    loss_noobj =tf.reduce_mean(tf.reduce_sum((1-y_true)*bce(y_true,y_pred)[:,tf.newaxis],axis=1))
    
    return loss_noobj

In [12]:
anchors =[[0.015,0.037],[0.043,0.104],[0.11,0.278],[0.351,0.66]]

In [13]:
model = TinyYOLOv3(1,anchor_boxes=anchors,train=True,mode = "finetuning")
model.build(batch_input_shape=(None,416,416,3))
print(model.load_weights("/home/sergio/TinyYOLOv3-Pedestrian-Detection/yolov3-tiny.weights"))

Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Pesos de la convolucion (432,)
CONV SHAPE (16, 3, 3, 3)
Pesos de la convolucion (4608,)
CONV SHAPE (32, 16, 3, 3)
Pesos de la convolucion (18432,)
CONV SHAPE (64, 32, 3, 3)
Pesos de la convolucion (73728,)
CONV SHAPE (128, 64, 3, 3)
Pesos de la convolucion (294912,)
CONV SHAPE (256, 128, 3, 3)
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
Pesos de la convolucion (4718592,)
CONV SHAPE (1024, 512, 3, 3)
Pesos de la convolucion (262144,)
CONV SHAPE (256, 1024, 1, 1)
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
Pesos de la convolucion (130560,)
CONV SHAPE (255, 512, 1, 1)
Pesos de la convolucion (32768,)
CONV SHAPE (128, 256, 1, 1)
Pesos de la convolucion (884736,)
CONV SHAPE (256, 384, 3, 3)
Pesos de la convolucion (65280,)
CONV SHAPE (255, 256, 1, 1)
8858734


In [15]:
tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Nadam(learning_rate=0.001)

#opt = tf.keras.optimizers.SGD(learning_rate=0.0000001, momentum=0.9, nesterov=True)

losses = {"output_1": loss_xy,
          "output_2": loss_wh,
          "output_3":loss_objectness,
          "output_4":loss_no_objectness
          
}

metrics = {"output_3":[Precision(),Recall(),TrueNegatives(0.5),TruePositives(0.5),FalseNegatives(0.5),FalsePositives(0.5)]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[2,1,4,1])
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")

#model._set_inputs((None,416,416,3))

In [16]:
checkpoint = tf.train.Checkpoint(optimizer=opt)
checkpoint_directory = "/home/sergio/Documents/optimizers_saved"
checkpoint_prefix = os.path.join(checkpoint_directory, "ckpt")

checkpoint.save(file_prefix=checkpoint_prefix)

'/home/sergio/Documents/optimizers_saved/ckpt-1'

In [17]:
print(opt._decayed_lr(tf.float32))
print(opt._iterations)
print(opt._m_cache)

tf.Tensor(0.001, shape=(), dtype=float32)
<tf.Variable 'iter:0' shape=() dtype=int64, numpy=0>
None


In [18]:
history = model.fit(train_dataset, epochs=25,validation_data=val_dataset)

Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Epoch 1/25
Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
   9213/Unknown - 579s 63ms/step - loss: 42.8893 - output_1_loss: 6.2392 - output_2_loss: 2.2733 - output_3_loss: 0.2197 - output_4_loss: 27.2587 - output_3_precision_1: 0.4851 - output_3_recall_1: 0.0791 - output_3_true_negatives_1: 123971224.0000 - output_3_true_positives_1: 42198.0000 - output_3_false_negatives_1: 491348.0000 - output_3_false_positives_1: 44792.0000Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
9213/9213 [==============================] - 588s 64ms/step - loss: 42.8893 - output_1_loss: 6.2392 - output_2_loss: 2.2733 - output_3_loss: 0.2197 - output_4_loss: 27.2587 - output_

In [19]:
model.save_weights('/home/sergio/Documents/models_saved/pesos_25_epochs_nadam_2_1_4_1_0dot001')

In [20]:
import json

os.chdir("/home/sergio/Documents/Checkpoints_experiements")

history_finetuning_good_bn_nadam_25_epochs_0dot001 = history.history
with open('history_finetuning_good_bn_nadam_25_epochs_0dot001.json', 'w') as outfile:
    json.dump(str(history_finetuning_good_bn_nadam_25_epochs_0dot001), outfile)

In [22]:
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")
history_part2 = model.fit(train_dataset, epochs=35,validation_data=val_dataset)

Epoch 1/35
9213/9213 [==============================] - 556s 60ms/step - loss: 24.2648 - output_1_loss: 3.5414 - output_2_loss: 1.1958 - output_3_loss: 0.1356 - output_4_loss: 15.4437 - output_3_precision_1: 0.7666 - output_3_recall_1: 0.4164 - output_3_true_negatives_1: 123947880.0000 - output_3_true_positives_1: 222232.0000 - output_3_false_negatives_1: 311446.0000 - output_3_false_positives_1: 67649.0000 - val_loss: 26.6281 - val_output_1_loss: 3.8407 - val_output_2_loss: 1.5473 - val_output_3_loss: 0.1538 - val_output_4_loss: 16.7841 - val_output_3_precision_1: 0.7553 - val_output_3_recall_1: 0.4112 - val_output_3_true_negatives_1: 5043607.0000 - val_output_3_true_positives_1: 9577.0000 - val_output_3_false_negatives_1: 13713.0000 - val_output_3_false_positives_1: 3103.0000
Epoch 2/35
9213/9213 [==============================] - 540s 59ms/step - loss: 24.0947 - output_1_loss: 3.5258 - output_2_loss: 1.2152 - output_3_loss: 0.1342 - output_4_loss: 15.2913 - output_3_precision_1: 0.7

In [23]:
model.save_weights('/home/sergio/Documents/models_saved/pesos_60_epochs_nadam_2_1_4_1_0dot001')

In [24]:
import json

os.chdir("/home/sergio/Documents/Checkpoints_experiements")

history_finetuning_good_bn_nadam_25_to_60_epochs_0dot001 = history_part2.history
with open('history_finetuning_good_bn_nadam_25_to_60_epochs_0dot001.json', 'w') as outfile:
    json.dump(str(history_finetuning_good_bn_nadam_25_to_60_epochs_0dot001), outfile)

In [ ]:
#Probando leer los pesos

In [85]:
anchors =[[0.015,0.037],[0.043,0.104],[0.11,0.278],[0.351,0.66]]
model_prueba = TinyYOLOv3(1,anchor_boxes=anchors,train=True,mode = "finetuning")
model_prueba.build(batch_input_shape=(None,416,416,3))
print(model_prueba.load_weights_darknet("/home/sergio/TinyYOLOv3-Pedestrian-Detection/yolov3-tiny.weights"))

Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Pesos de la convolucion (432,)
CONV SHAPE (16, 3, 3, 3)
Pesos de la convolucion (4608,)
CONV SHAPE (32, 16, 3, 3)
Pesos de la convolucion (18432,)
CONV SHAPE (64, 32, 3, 3)
Pesos de la convolucion (73728,)
CONV SHAPE (128, 64, 3, 3)
Pesos de la convolucion (294912,)
CONV SHAPE (256, 128, 3, 3)
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
Pesos de la convolucion (4718592,)
CONV SHAPE (1024, 512, 3, 3)
Pesos de la convolucion (262144,)
CONV SHAPE (256, 1024, 1, 1)
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
Pesos de la convolucion (130560,)
CONV SHAPE (255, 512, 1, 1)
Pesos de la convolucion (32768,)
CONV SHAPE (128, 256, 1, 1)
Pesos de la convolucion (884736,)
CONV SHAPE (256, 384, 3, 3)
Pesos de la convolucion (65280,)
CONV SHAPE (255, 256, 1, 1)
8858734


In [86]:
model_prueba.load_weights('/home/sergio/Documents/models_saved/pesos_60_epochs/pesos_60_epochs_nadam_2_1_4_1_0dot001')

In [87]:
tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Nadam(learning_rate=0.001)
#opt = tf.keras.optimizers.SGD(learning_rate=0.0000001, momentum=0.9, nesterov=True)

losses = {"output_1": loss_xy,
          "output_2": loss_wh,
          "output_3":loss_objectness,
          "output_4":loss_no_objectness
          
}

metrics = {"output_3":[Precision(),Recall(),TrueNegatives(0.5),TruePositives(0.5),FalseNegatives(0.5),FalsePositives(0.5)]}
model_prueba.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[2,1,4,1])
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")


In [88]:
model_prueba.fit(train_dataset, epochs=20,validation_data=val_dataset)

Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Epoch 1/20
Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
   9213/Unknown - 560s 61ms/step - loss: 18.3994 - output_1_loss: 2.8569 - output_2_loss: 1.0551 - output_3_loss: 0.1005 - output_4_loss: 11.2283 - output_3_precision_6: 0.8440 - output_3_recall_6: 0.6001 - output_3_true_negatives_5: 123956824.0000 - output_3_true_positives_5: 320130.0000 - output_3_false_negatives_5: 213350.0000 - output_3_false_positives_5: 59163.0000Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
9213/9213 [==============================] - 569s 62ms/step - loss: 18.3994 - output_1_loss: 2.8569 - output_2_loss: 1.0551 - output_3_loss: 0.1005 - output_4_loss: 11.2283 - output

In [66]:
history_part3_reseted_optimizer = model.fit(train_dataset, epochs=5,validation_data=val_dataset)

Epoch 1/5
Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
   9213/Unknown - 546s 59ms/step - loss: 25.4485 - output_1_loss: 3.6778 - output_2_loss: 1.2129 - output_3_loss: 0.1420 - output_4_loss: 16.3123 - output_3_precision_3: 0.7524 - output_3_recall_3: 0.3807 - output_3_true_negatives_3: 123949296.0000 - output_3_true_positives_3: 203099.0000 - output_3_false_negatives_3: 330447.0000 - output_3_false_positives_3: 66826.0000Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
9213/9213 [==============================] - 556s 60ms/step - loss: 25.4485 - output_1_loss: 3.6778 - output_2_loss: 1.2129 - output_3_loss: 0.1420 - output_4_loss: 16.3123 - output_3_precision_3: 0.7524 - output_3_recall_3: 0.3807 - output_3_true_negatives_3: 123949296.0000 - output_3_true

In [69]:
import json

os.chdir("/home/sergio/Documents/Checkpoints_experiements")

history_finetuning_good_bn_nadam_first_10_epochs_0dot001 = history.history
with open('history_finetuning_good_bn_nadam_first_10_epochs_0dot001.json', 'w') as outfile:
    json.dump(str(history_finetuning_good_bn_nadam_first_10_epochs_0dot001), outfile)

In [74]:
os.chdir("/home/sergio/Documents/Checkpoints_experiements")

history_finetuning_good_bn_nadam_second_10_epochs_0dot001 = history_part2.history
with open('history_finetuning_good_bn_nadam_second_10_epochs_0dot001.json', 'w') as outfile:
    json.dump(str(history_finetuning_good_bn_nadam_second_10_epochs_0dot001), outfile)

In [75]:
os.chdir("/home/sergio/Documents/Checkpoints_experiements")

history_finetuning_good_bn_nadam_next_5_epochs_0dot001_reset_nadam = history_part3_reseted_optimizer.history
with open('history_finetuning_good_bn_nadam_next_5_epochs_0dot001_reset_nadam.json', 'w') as outfile:
    json.dump(str(history_finetuning_good_bn_nadam_next_5_epochs_0dot001_reset_nadam), outfile)

In [62]:
print(opt._decayed_lr(tf.float32))
print(opt._iterations)
print(opt._m_cache)

tf.Tensor(1.3549401e-06, shape=(), dtype=float32)
<tf.Variable 'Nadam/iter:0' shape=() dtype=int64, numpy=184260>
<tf.Variable 'Nadam/momentum_cache:0' shape=() dtype=float32, numpy=0.0>


In [17]:
checkpoint.save(file_prefix=checkpoint_prefix)

'/home/sergio/Documents/optimizers_saved/ckpt-2'

In [30]:
to_restore = tf.keras.optimizers.Nadam(learning_rate=0.001)
print(opt._decayed_lr(tf.float32))
print(opt._iterations)

tf.Tensor(0.001, shape=(), dtype=float32)
<tf.Variable 'iter:0' shape=() dtype=int64, numpy=0>


In [42]:
opt.variables()

[<tf.Variable 'iter:0' shape=() dtype=int64, numpy=0>]

In [47]:
print(opt._m_cache)

None


In [35]:
new_root = tf.train.Checkpoint(opt=to_restore)
status = new_root.restore(tf.train.latest_checkpoint('/home/sergio/Documents/optimizers_saved')).assert_existing_objects_matched()

AssertionError: Some Python objects were not bound to checkpointed values, likely due to changes in the Python program: [<tf.Variable 'iter:0' shape=() dtype=int64, numpy=0>, <tf.Variable 'beta_1:0' shape=() dtype=float32, numpy=0.9>, <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.001>, <tf.Variable 'decay:0' shape=() dtype=float32, numpy=0.004>, <tf.Variable 'beta_2:0' shape=() dtype=float32, numpy=0.999>]

In [39]:
tf.train.list_variables(tf.train.latest_checkpoint('/home/sergio/Documents/optimizers_saved'))

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('optimizer/beta_1/.ATTRIBUTES/VARIABLE_VALUE', []),
 ('optimizer/beta_2/.ATTRIBUTES/VARIABLE_VALUE', []),
 ('optimizer/decay/.ATTRIBUTES/VARIABLE_VALUE', []),
 ('optimizer/iter/.ATTRIBUTES/VARIABLE_VALUE', []),
 ('optimizer/learning_rate/.ATTRIBUTES/VARIABLE_VALUE', []),
 ('optimizer/momentum_cache/.ATTRIBUTES/VARIABLE_VALUE', []),
 ('save_counter/.ATTRIBUTES/VARIABLE_VALUE', [])]

In [32]:
status

In [34]:
print(to_restore._decayed_lr(tf.float32))
print(to_restore._iterations)

tf.Tensor(0.001, shape=(), dtype=float32)
<tf.Variable 'iter:0' shape=() dtype=int64, numpy=0>


In [37]:
history_finetuning_good_bn_nadam_10_epochs_0dot001 = history.history

In [38]:
import json
import os

os.chdir("/home/sergio/Documents/Checkpoints_experiements")

with open('history_finetuning_good_bn_nadam_10_epochs_0dot001.json', 'w') as outfile:
    json.dump(str(history_finetuning_good_bn_nadam_10_epochs_0dot001), outfile)

In [8]:
model_save_path = "/home/sergio/Documents/models_saved/modelo_prueba"

custom_objects = {"TinyYOLOv3":TinyYOLOv3,"losses":losses,"loss_xy":loss_xy,"loss_wh":loss_wh,"loss_objectness":loss_objectness,"loss_no_objectness":loss_no_objectness}
          
tf.keras.models.save_model(model, model_save_path, include_optimizer=True)

NameError: name 'losses' is not defined

In [27]:
new_model = tf.keras.models.load_model('/home/sergio/Documents/models_saved/modelo_prueba',custom_objects,tf.TensorSpec(shape=(None, 416, 416, 3), dtype=tf.float32, name='inputs'))

ValueError: Could not find matching function to call loaded from the SavedModel. Got:
  Positional arguments (2 total):
    * Tensor("inputs:0", shape=(None, 416, 416, 3), dtype=float32)
    * Tensor("training:0", shape=(), dtype=bool)
  Keyword arguments: {}

Expected these arguments to match one of the following 4 option(s):

Option 1:
  Positional arguments (2 total):
    * TensorSpec(shape=(None, 416, 416, 3), dtype=tf.float32, name='input_1')
    * False
  Keyword arguments: {}

Option 2:
  Positional arguments (2 total):
    * TensorSpec(shape=(None, 416, 416, 3), dtype=tf.float32, name='inputs')
    * False
  Keyword arguments: {}

Option 3:
  Positional arguments (2 total):
    * TensorSpec(shape=(None, 416, 416, 3), dtype=tf.float32, name='inputs')
    * True
  Keyword arguments: {}

Option 4:
  Positional arguments (2 total):
    * TensorSpec(shape=(None, 416, 416, 3), dtype=tf.float32, name='input_1')
    * True
  Keyword arguments: {}

In [8]:
import sys
sys.path.append("/home/sergio/TinyYOLOv3-Pedestrian-Detection")

from YOLOblocks import TinyYOLOv3,BasicBlock,PredictionLayer#,YOLOLossBasicBlock
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
#from tensorflow.python.tools import freeze_graph
#from skimage.io import imread,imshow
#from skimage.transform import resize 
import time
#from tensorflow.compat.v1.image import decode_image
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import os

In [9]:
a =BasicBlock(num_filters = 16, kernel_size = 3,name="BasicBlock1",trainable=True)

In [10]:
anchors =[[0.015,0.037],[0.043,0.104],[0.11,0.278],[0.351,0.66]]
b = PredictionLayer(anchors[len(anchors)//2:],conf_thresh=0.5,grid_size=13,num_classes=1,name="Prediction1",training=True)

In [11]:
model = TinyYOLOv3(1,anchor_boxes=anchors,train=True,mode = "finetuning")
model.build(batch_input_shape=(None,416,416,3))
print(model.load_weights("/home/sergio/TinyYOLOv3-Pedestrian-Detection/yolov3-tiny.weights"))

Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Pesos de la convolucion (432,)
CONV SHAPE (16, 3, 3, 3)
Pesos de la convolucion (4608,)
CONV SHAPE (32, 16, 3, 3)
Pesos de la convolucion (18432,)
CONV SHAPE (64, 32, 3, 3)
Pesos de la convolucion (73728,)
CONV SHAPE (128, 64, 3, 3)
Pesos de la convolucion (294912,)
CONV SHAPE (256, 128, 3, 3)
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
Pesos de la convolucion (4718592,)
CONV SHAPE (1024, 512, 3, 3)
Pesos de la convolucion (262144,)
CONV SHAPE (256, 1024, 1, 1)
Pesos de la convolucion (1179648,)
CONV SHAPE (512, 256, 3, 3)
Pesos de la convolucion (130560,)
CONV SHAPE (255, 512, 1, 1)
Pesos de la convolucion (32768,)
CONV SHAPE (128, 256, 1, 1)
Pesos de la convolucion (884736,)
CONV SHAPE (256, 384, 3, 3)
Pesos de la convolucion (65280,)
CONV SHAPE (255, 256, 1, 1)
8858734


In [12]:
model.get_config()

{'train': True,
 'num_classes': 1,
 'num_anchors': 4,
 'filter_prediction_layer': 10,
 'block1': <YOLOblocks.BasicBlock at 0x7f87b4302128>,
 'block2': <YOLOblocks.BasicBlock at 0x7f87b4302a58>,
 'block3': <YOLOblocks.BasicBlock at 0x7f87b424dbe0>,
 'block4': <YOLOblocks.BasicBlock at 0x7f87b424d278>,
 'block5': <YOLOblocks.BasicBlock at 0x7f87b42c9550>,
 'block6': <YOLOblocks.BasicBlock at 0x7f87b42c95f8>,
 'block7': <YOLOblocks.BasicBlock at 0x7f87b42dd4a8>,
 'block8': <YOLOblocks.BasicBlock at 0x7f87b42dd2b0>,
 'block9': <YOLOblocks.BasicBlock at 0x7f87b42dde80>,
 'block10': <YOLOblocks.BasicBlock at 0x7f87b4228400>,
 'block11': <YOLOblocks.BasicBlock at 0x7f87b42285f8>,
 'block12': <YOLOblocks.BasicBlock at 0x7f87b4228d30>,
 'block13': <YOLOblocks.BasicBlock at 0x7f87b4302518>,
 'concat_block': <tensorflow.python.keras.layers.merge.Concatenate at 0x7f87b423e278>,
 'upsamp': <tensorflow.python.keras.layers.convolutional.UpSampling2D at 0x7f87b423e2e8>,
 'yolo1': <YOLOblocks.Predictio

In [13]:
tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Nadam(learning_rate=0.001)
#opt = tf.keras.optimizers.SGD(learning_rate=0.0000001, momentum=0.9, nesterov=True)

losses = {"output_1": loss_xy,
          "output_2": loss_wh,
          "output_3":loss_objectness,
          "output_4":loss_no_objectness
          
}

metrics = {"output_3":[Precision(),Recall(),TrueNegatives(0.5),TruePositives(0.5),FalseNegatives(0.5),FalsePositives(0.5)]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[2,1,4,1])
os.chdir("/home/sergio/Documents/pedestrian_dataset_train_tfr")


In [14]:
history = model.fit(train_dataset, epochs=10,validation_data=val_dataset)

Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Epoch 1/10
Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
    235/Unknown - 19s 81ms/step - loss: 134.0872 - output_1_loss: 10.1854 - output_2_loss: 2.9031 - output_3_loss: 0.3720 - output_4_loss: 109.3252 - output_3_precision: 0.0023 - output_3_recall: 0.0107 - output_3_true_negatives: 3097573.0000 - output_3_true_positives: 126.0000 - output_3_false_negatives: 11624.0000 - output_3_false_positives: 54357.0000

KeyboardInterrupt: 

In [15]:
model.save_weights('/home/sergio/Documents/models_saved/pesos_prueba')

In [33]:
model_save_path = "/home/sergio/Documents/models_saved/modelo_prueba"
#tf.keras.models.save_model(model, model_save_path, include_optimizer=True)
tf.saved_model.save(model, model_save_path)

Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
Se redimensiona la salida de la CNN a: (None, 338, 5)
Se redimensiona la salida de la CNN a: (None, 1352, 5)
INFO:tensorflow:Assets written to: /home/sergio/Documents/models_saved/modelo_prueba/assets


In [35]:
#tf.TensorSpec(shape=(None, 416, 416, 3), dtype=tf.float32, name='inputs')
#,"loss_xy":loss_xy,"loss_wh":loss_wh,"loss_objectness":loss_objectness,"loss_no_objectness":loss_no_objectness
#custom_objects = {"TinyYOLOv3":TinyYOLOv3,"BasicBlock":BasicBlock,"PredictionLayer":PredictionLayer,"losses":losses}
custom_objects = {"loss_xy":loss_xy,"loss_wh":loss_wh,"loss_objectness":loss_objectness,"loss_no_objectness":loss_no_objectness} 

new_model = tf.keras.models.load_model(filepath='/home/sergio/Documents/models_saved/modelo_prueba',custom_objects=custom_objects,compile=False)

In [38]:
history = new_model.fit(train_dataset, epochs=10,validation_data=val_dataset)

AttributeError: 'TinyYOLOv3' object has no attribute 'sample_weight_mode'

In [54]:
opt = tf.keras.optimizers.Nadam(learning_rate=0.001)

In [55]:
checkpoint = tf.train.Checkpoint(optimizer=opt)

In [56]:
checkpoint_directory = "/home/sergio/Documents/optimizers_saved"
checkpoint_prefix = os.path.join(checkpoint_directory, "ckpt")

checkpoint.save(file_prefix=checkpoint_prefix)

'/home/sergio/Documents/optimizers_saved/ckpt-1'